<img alt="Cover" src="./images/1.cover.png" width="3000" />

---

# What? Why?

In [1]:
fun quickSort(collection: List<Int>) {  }

In [2]:
quickSort(listOf(1, 2, 3)) // OK

In [3]:
quickSort(listOf(1.0, 2.0, 3.0)) // NOT OK

Line_3.jupyter.kts (1:11 - 32) Type mismatch: inferred type is List<Double> but List<Int> was expected

In [4]:
fun quickSort(collection: List<Double>) {  } // overload (we’ll get back to this a bit later)

In [5]:
quickSort(listOf(1.0, 2.0, 3.0)) // OK

In [6]:
quickSort(listOf(1, 2, 3)) // OK

<font size="5">Kotlin `Number` inheritors: `Int`, `Double`, `Byte`, `Float`, `Long`, `Short`. Do we need 4 more implementations of quickSort?</font>

---

# How?

<font size="5">Does the quickSort algorithm actually care what is it sorting? No, as long as it can compare two values against each other.</font>

In [7]:
fun <T : Comparable<T>> quickSort(collection: Collection<T>): Collection<T> = collection // We will not implement it

In [8]:
quickSort(listOf(1.0, 2.0, 3.0)) // OK

[1.0, 2.0, 3.0]

In [9]:
quickSort(listOf(1, 2, 3)) // OK

[1, 2, 3]

In [10]:
quickSort(listOf("one", "two", "three")) // OK

[one, two, three]

<font size="5">Generics allow you to write code that can work with any type or with types that should satisfy some rules (constraints) but are not limited in any other ways: type parameters.</font>


In [11]:
data class Holder<T>(val value: T)

In [12]:
val intHolder = Holder<Int>(23)
println(intHolder)

Holder(value=23)


In [13]:
val cupHolder = Holder("cup") // Generic parameter type can be inferred
println(cupHolder)

Holder(value=cup)


---

# Constraints

<font size="5">Sometimes we do not want to work with an arbitrary type and expect it to provide us with some functionality. In such cases type constraints in the form of upper bounds are used: upper bounds.</font>

In [14]:
import org.jetbrains.kotlin.public.course.generics.Movable

class Pilot<T : Movable>(val vehicle: T) {
    fun go() { vehicle.move() }
}

In [15]:
import org.jetbrains.kotlin.public.course.generics.Car

val ryanGosling = Pilot<Car>(Car("Chevy", "Malibu"))
ryanGosling.go()

Car is moving


In [16]:
import org.jetbrains.kotlin.public.course.generics.Plane

val sullySullenberger = Pilot<Plane>(Plane("Airbus", "A320"))
sullySullenberger.go()

Plane is moving


---

# Constraints continued

<font size="5">There can be several parameter types, and generic classes can participate in inheritance.</font>

In [17]:
public interface MutableMap<K, V> : Map<K, V> {  }

<font size="5">There can also be several constraints (which means the type parameter has to implement several interfaces):</font>

In [18]:
import org.jetbrains.kotlin.public.course.generics.Awesome

fun <T, S> moveInAnAwesomeWayAndCompare(a: T, b: S) where T : Comparable<T>, S : Comparable<T>, T : Awesome, T : Movable {  }

---

# Star-projection

<font size="5">When you do not care about the parameter type, you can use star-projection `*`.</font>

In [19]:
fun printKeys(map: MutableMap<*, *>) {  }

---

# Let's go back

In [20]:
open class A
open class B : A()
class C : B()

// Nothing <: C <: B <: A <: Any

<font size="5">This means that the `Any` class is the _superclass_ for all the classes and at the same time `Nothing` is a _subtype_ of any type</font>

<font size="5">Consider a basic example:</font>

In [21]:
interface HolderExample<T> {
    fun push(newValue: T) // consumes an element
    
    fun pop(): T // produces an element
    
    fun size(): Int // does not interact with T
}

<font size="5">In Kotlin there are type projections:</font>

<font size="5">`G<T>` // invariant, can **consume** and **produce** elements </font>
<font size="5">`G<in T>` // contravariant, can only **consume** elements </font>
<font size="5">`G<out T>` // covariant, can only **produce** elements </font>
<font size="5">`G<*>` // star-projection, does not interact with T</font>

---

# Several examples

<font size="5">`G<T>` // invariant, can **consume** and **produce** elements </font>

In [22]:
interface HolderSimple<T> {
     fun push(newValue: T)// consumes an element: OK
     
     fun pop(): T // produces an element: OK
     
     fun size(): Int // does not interact with T: OK
}

<font size="5">`G<in T>` // contravariant, can only **consume** elements </font>

In [23]:
interface HolderConsumer<in T> {
     fun push(newValue: T) // consumes an element: OK
     
     fun pop(): T // produces an element: ERROR: [TYPE_VARIANCE_CONFLICT_ERROR] Type parameter T is declared as 'in' but occurs in 'out' position in type T
    
     fun size(): Int // does not interact with T: OK
}

Line_23.jupyter.kts (4:17 - 18) Type parameter T is declared as 'in' but occurs in 'out' position in type T

<font size="5">`G<out T>` // covariant, can only **produce** elements </font>

In [24]:
interface HolderProducer<out T> {
     fun push(newValue: T) // consumes an element: ERROR: [TYPE_VARIANCE_CONFLICT_ERROR] Type parameter T is declared as 'out' but occurs in 'in' position in type T
     fun pop(): T // produces an element: OK
     fun size(): Int // does not interact with T: OK
}

Line_24.jupyter.kts (2:25 - 26) Type parameter T is declared as 'out' but occurs in 'in' position in type T

<font size="5">`G<*>` // star-projection, does not interact with T</font>

In [25]:
fun <T> foo1(holder: HolderSimple<T>, t: T) {
     holder.push(t) // OK
}

In [26]:
fun foo2(holder: HolderSimple<Any>, t: Any) {
    holder.push(t) // OK
}

In [27]:
fun <T> foo3(holder: HolderSimple<*>, t: T) {
     holder.push(t) // ERROR: [TYPE_MISMATCH] Type mismatch. Required: Nothing. Found: T
}

Line_27.jupyter.kts (2:18 - 19) Type mismatch: inferred type is T but Nothing was expected

---

# Subtyping

<img alt="Cover" src="./images/2.png" width="3000" />

---

<img alt="Cover" src="./images/3.png" width="3000" />

In [28]:
val c: C = C()
val b: B = c // C <: B, OK

In [29]:
val holderC: Holder<C> = Holder(C())
val holderB: Holder<B> = holderC // ERROR: Type mismatch. Required: Holder<B>. Found: Holder<C>.

Line_29.jupyter.kts (2:26 - 33) Type mismatch: inferred type is Line_11_jupyter.Holder<Line_20_jupyter.C> but Line_11_jupyter.Holder<Line_20_jupyter.B> was expected

In [ ]:
// BUT
val holderB: Holder<B> = Holder(C()) // OK, because of casting

---

# Subtyping: Next

In [ ]:
class HolderSubtypingDemo<T> (var value: T?) {
   fun pop(): T? = value.also { value = null }
   fun push(newValue: T?): T? = value.also { value = newValue }
   fun steal(other: HolderSubtypingDemo<T>) { value = other.pop() }
   fun gift(other: HolderSubtypingDemo<T>) { other.push(pop()) }
}

<font size="5">HolderSubtypingDemo<Nothing> !<:> HolderSubtypingDemo<C> !<:> HolderSubtypingDemo<B> !<:> HolderSubtypingDemo<A> !<:> HolderSubtypingDemo<Any></font>

In [ ]:
val holderB: HolderSubtypingDemo<B> = HolderSubtypingDemo(B())
val holderA: HolderSubtypingDemo<A> = HolderSubtypingDemo(null)
holderA.steal(holderB) // ERROR: Type mismatch. Required: Holder<A>. Found: Holder<B>.
holderB.gift(holderA) // ERROR: Type mismatch. Required: Holder<B>. Found: Holder<A>.

---

# Type projection: in

In [ ]:
class HolderIn<T> (var value: T?) {
   fun pop(): T? = value.also { value = null }
   fun push(newValue: T?): T? = value.also { value = newValue }
    
   fun gift(other: HolderIn<in T>) { other.push(pop()) }
}

In [ ]:
val holderB: HolderIn<B> = HolderIn(B())
val holderA: HolderIn<A> = HolderIn(null)
holderB.gift(holderA) // OK

<font size="5">Type projection: `other` is a restricted (projected) generic. You can only call methods that **accept** the type parameter `T`, which in
this case means that you can only call `push()`.</font>

<font size="5">This is contravariance:</font>

<font size="5">`Nothing <: C <: B <: A <: Any`</font>
<font size="5">`HolderIn<Nothing> :> HolderIn<C> :> HolderIn<B> :> HolderIn<A> :> HolderIn<Any>`</font>

---

# Type projection: out

In [ ]:
class HolderOut<T> (var value: T?) {
   fun pop(): T? = value.also { value = null }
   fun push(newValue: T?): T? = value.also { value = newValue }

   fun steal(other: HolderOut<out T>) { value = other.pop() }
}

In [ ]:
val holderB: HolderOut<B> = HolderOut(B())
val holderA: HolderOut<A> = HolderOut(null)
holderA.steal(holderB) // OK

<font size="5">Type projection: `other` is a restricted (projected) generic. You can only call methods that **return** the type parameter `T`, which in
this case means that you can only call `pop()`.</font>

<font size="5">This is covariance:</font>

<font size="5">`Nothing <: C <: B <: A <: Any`</font>
<font size="5">`HolderIn<Nothing> <: HolderIn<C> <: HolderIn<B> <: HolderIn<A> <: HolderIn<Any>`</font>

---

# Type projections

In [ ]:
class HolderFinalDemo<T> (var value: T?) {
   fun pop(): T? = value.also { value = null }
   fun push(newValue: T?): T? = value.also { value = newValue }
    
    
   fun steal(other: HolderFinalDemo<out T>) {
      val oldValue = push(other.pop())
      other.push(oldValue) // ERROR: Type mismatch. Required: Nothing?. Found: T?.
   }
   fun gift(other: HolderFinalDemo<in T>) {
      val otherValue = other.push(pop())
      push(otherValue) // ERROR: Type mismatch. Required: T?. Found: Any?.
   }
}

<font size="5">`out T` returns something that can be cast to `T` and accepts literally `Nothing`.</font>

<font size="5">`in T` accepts something that can be cast to `T` and returns a meaningless `Any?`.</font>

---

# Type erasure

<font size="5">At runtime, the instances of generic types do not hold any information about their actual type arguments. The type information is said to be erased. The same byte-code is used in all usages of the generic as opposed to C++, where each template is compiled separately for each type parameter provided.</font>

<font size="5">- Any `MutableMap<K, V>` becomes `MutableMap<*, *>` in the runtime. (Actually, in the **Kotlin/JVM** runtime we have just `java.util.Map` to preserve compatibility with Java.)</font>
<font size="5">- Any `Pilot<T : Movable>` becomes `Pilot<Movable>`.</font>

<font size="5">As a corollary, you cannot override a function (in **Kotlin/JVM**) by changing generic type parameters:</font>

In [ ]:
fun quickSort(collection: Collection<Int>) {  }
fun quickSort(collection: Collection<Double>) {  }

<font size="5">Both become `quickSort(collection: Collection<*>)` and their signatures clash.</font>

<font size="5">But you can use the `JvmName` annotation:</font>

In [ ]:
@JvmName("quickSortInt")
fun quickSort(collection: Collection<Int>) {  } 
fun quickSort(collection: Collection<Double>) {  }

---

# Nullability in generics

<font size="5">Contrary to common sense, in Kotlin a type parameter specified as `T` can be nullable.</font>

In [ ]:
class HolderNullability<T>(val value: T) {  } // Notice there is no `?`
val holderA: HolderNullability<A?> = HolderNullability(null) // T = A? and that is OK

<font size="5">To prohibit such behavior, you can use a non-nullable `Any` as a constraint.</font>

In [ ]:
class HolderNotNull<T : Any>(val value: T) {  }
val holderA: HolderNotNull<A?> = HolderNotNull(null) // ERROR: Type argument is not within its bounds. Expected: Any. Found: A?.

<font size="5">You may also find intersection helpful:</font>

In [ ]:
fun <T> elvisLike(x: T, y: T & Any): T & Any = x ?: y // T & Any is populated with all values from T besides null

---

# Inline functions

<font size="5">If they are used as first-class objects, functions are stored as objects, thus requiring memory allocations, which introduce runtime overhead.</font>

In [ ]:
fun foo(str: String, call: (String) -> Unit) {
     call(str)
}

In [ ]:
foo("Top level function with lambda example") { print(it) }

<img alt="Cover" src="./images/4.png" width="3000" />

<img alt="Cover" src="./images/5.png" width="3000" />

<img alt="Cover" src="./images/6.png" width="3000" />

<font size="5">`inline` affects not only the function itself, but also all the lambdas passed as arguments.
If you do not want some of the lambdas passed to an `inline` function to be inlined (_for example, inlining large functions is not recommended_), you can mark some of the function parameters with the `noinline` modifier.</font>

In [ ]:
inline fun fooNoInlineArg(str: String, call1: (String) -> Unit, noinline call2: (String) -> Unit) { 
    call1(str) // Will be inlined
    call2(str) // Will not be inlined
}

<font size="5">You can use `return` in inlined lambdas, this is called _non-local return_, which can lead to unexpected behaviour:</font>

In [8]:
inline fun fooNonLocalReturn(call1: () -> Unit, call2: () -> Unit) {
    call1()
    call2() 
}

In [10]:
fun main() {
    println("Step#1")
    fooNonLocalReturn({
        println("Step#2")
        return
    }, { println("Step#3") })
    println("Step#4")
}

main()

Step#1
Step#2


<font size="5">To prohibit returning from the lambda expression we can mark the lambda as `crossinline`:</font>

In [ ]:
inline fun crossInlineFoo(crossinline call1: () -> Unit, call2: () -> Unit) {
    call1()
    call2() 
}

In [ ]:
println("Step#1")
crossInlineFoo({
    println("Step#2")
     return
}, // ERROR: 'return' is not allowed here
    { println("Step#3") })
println("Step#4")

In [ ]:
// BUT it works

println("Step#1")
crossInlineFoo({
    println("Step#2")
     return@crossInlineFoo
}, // ERROR: 'return' is not allowed here
    { println("Step#3") })
println("Step#4")

<font size="5">`crossinline` is especially useful when the lambda from an `inline` function is being called from another context, for example, if it is used to instantiate a `Runnable`:</font>

In [1]:
import kotlin.concurrent.thread

inline fun drive(crossinline specialCall: (String) -> Unit, call: (String) -> Unit) { 
    val nightCall = Runnable { specialCall("There's something inside you") }
    call("I'm giving you a nightcall to tell you how I feel")
    thread { nightCall.run() }
    call("I'm gonna drive you through the night, down the hills")
}

In [2]:
// Switch to Crossinline2.kt (an Notebook bug)

drive({ System.err.println(it) }) { println(it) }

Backend Internal error: Exception during IR lowering
File being compiled: Line_2.jupyter.kts
The root cause java.lang.RuntimeException was thrown at: org.jetbrains.kotlin.backend.jvm.codegen.FunctionCodegen.generate(FunctionCodegen.kt:51)

---

# Inline reified functions

<font size="5">Sometimes you need to access a type passed as a parameter:</font>

In [6]:
import org.jetbrains.kotlin.public.course.generics.Animal

fun <T: Animal> fooInlineError() {
    println(T::class) // ERROR: Cannot use 'T' as reified type parameter. Use a class instead---> add a param: t: KClass<T>
}


Line_6.jupyter.kts (3:9 - 15) 'Animal' is a final type, and thus a value of the type parameter is predetermined
Line_6.jupyter.kts (4:13 - 21) Cannot use 'T' as reified type parameter. Use a class instead.

<font size="5">You can use the `reified` keyword with `inline` functions:</font>

In [11]:
import org.jetbrains.kotlin.public.course.generics.Animal

inline fun <reified T: Animal> fooInlineOk() {
    println(T::class) // OK 
}

In [12]:
fooInlineOk<Animal>()

class org.jetbrains.kotlin.public.course.generics.Animal


<font size="5">Note that the compiler has to be able to know the actual type passed as a type argument so that it can modify the generated bytecode to use the corresponding class directly.</font>